<a href="https://colab.research.google.com/github/LeibGit/-DI_Bootcamp/blob/main/exercizexp_week8_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu
import numpy as np
import pandas as pd
import faiss
import uuid
import json
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer, InputExample
import chromadb
from chromadb.config import Settings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
pdf = pd.read_csv("labelled_newscatcher_dataset.csv", sep=';')
pdf.head()

In [ ]:
pdf.index = range(1, len(pdf) +1)
pdf.index.name = "id"
pdf.head()

In [ ]:
display(pdf)

In [ ]:
sub_ds = pdf[:1_000]
display(sub_ds)

In [ ]:
from sentence_transformers import InputExample

def example_create_fn(row):
  return InputExample(
      texts=[row["title"]]
  )

examples = sub_ds.apply(lambda x: example_create_fn(x), axis=1).tolist()
examples[:10]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

titles_list = sub_ds["title"].tolist()

titles_embedding = model.encode(titles_list)

len(titles_embedding), len(titles_embedding[0])

In [ ]:
id_index = sub_ds.index
faiss.normalize_L2(titles_embedding)

index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(titles_embedding[0])))
index_content.add_with_ids(titles_embedding, id_index)

In [ ]:
def search_content(query, pdf_to_index, k=3):
    query_vector = model.encode([query]) # Encode the query string into an embedding vector.
    faiss.normalize_L2(query_vector)  # Normalize the query vector.

    # Perform the search
    top_k = k
    similarities, indices = index_content.search(query_vector, top_k)  # Similarity scores and indices for the matches.

    results = pdf_to_index.loc[indices.flatten()]  # Retrieve the matching articles from pdf_to_index.
    results["similarities"] = similarities.flatten()  # Add similarity scores.
    return results

display(search_content("animal", sub_ds, k=5))

In [ ]:
chroma_client = chromadb.Client()
collection_name = "my_news"

# If a collection with the same name exists, delete it to avoid conflicts
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)

print(f"Creating collection: '{collection_name}'")
collection = chroma_client.create_collection(name=collection_name)

In [ ]:
# Display the DataFrame subset (for reference)
display(sub_ds)

collection.add(
    documents=sub_ds["title"][:100].tolist(),
    metadatas=[{"topic": topic} for topic in sub_ds["topic"][:100].tolist()],
    ids=[str(id) for id in sub_ds.index[:100].tolist()]
)

In [ ]:
import json

results = collection.query(
    query_texts=["space development"],
    n_results=2
)

print(json.dumps(results, indent=4))

In [ ]:
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
)

In [ ]:
question = "What's the latest news on space development?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"

In [ ]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])